In [3]:
import re
import time
import datetime
import pandas as pd

In [9]:
def merge_functions(text):
    # Remove strings specified in the pattern from the text
    #pattern = r'Physician|PCP|Assistants|Primary|Doctor|Care|-'
    #text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Remove two words that come after "Agent" (case-insensitive match)
    text = re.sub(r'\b[Aa]gent\b(?:\s+\w+){1,3}', 'agent', text, flags=re.IGNORECASE)

    # Remove two words that come after "agent" (case-insensitive match)
    text = re.sub(r'\bagent\b(?:\s+\w+){1,3}', 'agent', text, flags=re.IGNORECASE)

    # Remove strings with the format "Agent, [Name]" or "agent, [Name]"
    text = re.sub(r'\b[Aa]gent\s*,\s*[A-Za-z]+\s+[A-Za-z]+\b', '', text)

    # Remove strings with the format "Agent, [Name]" or "agent, [Name]"
    text = re.sub(r'\b[Rr]epresentative\s*,\s*[A-Za-z]+\s+[A-Za-z]+\b', '', text)

    text = re.sub(r'\b[Rr]epresentative\s*\b[A-Za-z]+\s+[A-Za-z]+\b', '', text)
    
    text = re.sub(r'\bagent\b(\s+\w+){1,3}', '', text, flags=re.IGNORECASE)


    #Dr. Alex Souze (PID#
    # Remove strings with this format: "[Name] [Last Name] (PID", except when preceded by "Dr."
    #text = re.sub(r'(?<!\bDr\.)\b[A-Za-z]+\s+[A-Za-z]+\s*\(PID[# ]\s*\d+\)', '', text)
    pattern2 = r'Dr\.\s+[A-Za-z]+\s+[A-Za-z]+\s*\(PID'

    #text = re.sub(pattern2, r'Dr. \g<0> hahaha', text)
    text = re.sub(pattern2, r'Dr. \g<0> xxxx', text)

    # Remove strings with this format: "[Name] [Last Name] (PID"
    text = re.sub(r'\b[A-Za-z]+\s+[A-Za-z]+\s*\(PID[# ]\s*\d+\)', '', text)
    
    # Remove strings with this format: "[Name] [Last Name], NPN"
    text = re.sub(r'\b[A-Za-z]+\s+[A-Za-z]+\s*,\s*\bNPN\b', '', text)

    # Remove strings with this format: "[Name] [Last Name], [string] agent"
    text = re.sub(r'\b[A-Za-z]+\s+[A-Za-z]+,\s+[A-Za-z]+\s+agent\b', '', text)

    # Remove strings with this format:  mother , [string that can include dot] [Name] [Surname]
    text = re.sub(r'\bmother\s*,\s*[A-Za-z.]+\s+[A-Za-z]+\s+[A-Za-z]+\b', '', text)
    text = re.sub(r'\bmother\s*,\s*[A-Za-z]*\.?\s+[A-Za-z]+\s+[A-Za-z]+\b', '', text)

    # Remove strings with this format:  daughter , [string that can include dot] [Name] [Surname]  daughter , Anu Joseph
    text = re.sub(r'\bdaughter\s*,\s*[A-Za-z]*\.?\s+[A-Za-z]+\s+[A-Za-z]+\b', '', text)

    # Clean stopwords from the text
   # stop_words = set(stopwords.words('english'))
   # words = word_tokenize(text)
   # filtered_words = [word for word in words if word.lower() not in stop_words]
    text = ' '.join(text)

    return text.strip()

In [10]:
df = pd.read_excel("data.xlsx")

In [23]:
def extract_doctor_names(notes):
  """Extracts doctor names from a string of text.

  Args:
    notes: A string of text containing doctor names.

  Returns:
    A string of doctor names, separated by commas.
  """

  doctor_names = []
  for match in re.finditer(r"\b[A-Z][a-z]+\b", notes):
    doctor_names.append(match.group())
  return ",".join(doctor_names)



In [19]:
df["Providers"] = df["notes"].apply(extract_doctor_names)
df["Providers"]

0             Physician,Amy,Joker,Instead,Dr,Musa,Anter
1     Doctor,David,Coleman,Agent,Kerim,Sahin,But,Som...
2                                           Kazim,Kurdi
3                  Adam,Smith,Mark,First,Doctor,Coleman
4                          Ebbie,Dark,She,Foresnjs,Adam
5     Primary,Sara,Lopez,Mary,Colly,After,Dr,Alex,Fe...
6     Orthopedic,Lopez,Tunner,Some,Doctor,Koray,Peks...
7     Endocrinologist,Gerry,Samantha,Adam,Boyler,You...
8                                  Podiatrist,Roy,Rolls
9                                       In,Shiv,Reed,We
10    Trump,Boyler,Dr,Scott,Hahaha,Before,Doctor,Jac...
11           Paul,Andrew,Primary,Care,Physician,But,The
12                                  As,Dr,Lewwis,Donald
13    Agent,Mary,Eric,Marcia,Tunner,Physician,Assist...
14    Agent,Erin,Shea,Hays,Gina,Fisher,Physician,Ass...
15                                       Dr,Golly,Adams
16    Physician,Amy,Joker,Agent,Kerim,Sahin,But,Dr,M...
17    Kazim,Kurdi,Adam,Smith,Mark,Literal,Doctor

In [103]:
def extract_provider_names_2(text):
    provider_names = []
    pattern = r"((Dr\.)|(PCP,)|(Doctor)|(PA)|(provider,)|(surgeon,)|(MD,)|(NP,)|(ist,))\s\b[A-Z].+?\b\s\b[A-Z].+?\b"
    matches = re.finditer(pattern, text)
    for match in matches:
        #print(match)
        
        provider_names.append(match.group(0))
    return provider_names



In [118]:
def extract_provider_names_3(text):
    provider_names = []
    pattern = r"((Dr\.?)|(PCP,)|(Doctor)|(PA)|(provider,)|(surgeon,)|(MD,)|(NP,)|(ist,))\s[A-Z][a-z-?]+\s[A-Z][a-z-?]+\s"
    matches = re.finditer(pattern, text)
    for match in matches:
        #print(match)
        provider_names.append(match.group(0))
        
        
    
    pattern = r"([A-Z][a-z-?]+\s)?[A-Z][a-z-?]+\s[A-Z][a-z-?]+,?\s(MD|NP)"
    matches = re.finditer(pattern, text)
    for match in matches:
        #print(match)
        provider_names.append(match.group(0))
    
    return provider_names


In [119]:
df["Providers3"] = df["notes"].apply(extract_provider_names_3)
df[["notes","Providers","Providers3"]]

,notes,Providers,Providers3
0,"Physician PCP, Amy Joker is a very good person...","[PCP, Amy Joker, Dr. Musa Anter]","[PCP, Amy Joker , Dr. Musa Anter ]"
1,"Doctor David Coleman, as the PC, I like that d...","[Doctor David Coleman, PA Kazick Downski]",[PA Kazick Downski ]
2,Kazim Kurdi as her PCP is very famous.,[],[]
3,PA Adam Smith drives a lot of different cars w...,[PA Adam Smith],[PA Adam Smith ]
4,PA Ebbie Dark always gets up early. She loves ...,"[PA Ebbie Dark, PA Foresnjs Adam]","[PA Ebbie Dark , PA Foresnjs Adam ]"
5,"Primary care provider, Sara-Lopez Mary-Colly, ...","[provider, Sara-Lopez Mary, Dr. Alex Ferguson,...","[Dr. Alex Ferguson , Dr. Special Doctor ]"
6,"Orthopedic surgeon, Lopez C. Tunner, MD, (PID:...","[surgeon, Lopez C. , Doctor Koray Peksever]",[Doctor Koray Peksever ]
7,"Endocrinologist, Gerry Samantha, NP, (PID: 243...","[ist, Gerry Samantha, Dr. Sarah Goray, PA Fore...","[Dr. Sarah Goray , PA Foresnjs Adam , Gerry Sa..."
8,"Podiatrist, Roy Rolls (NPI::23426234398) lets ...","[ist, Roy Rolls]","[ist, Roy Rolls ]"
9,"In the morning MD, Shiv Reed, PA-C, (PID:24243...","[MD, Shiv Reed]",[]


In [120]:
df["Providers"] = df["notes"].apply(extract_provider_names_2)
df[["notes","Providers"]]

,notes,Providers
0,"Physician PCP, Amy Joker is a very good person...","[PCP, Amy Joker, Dr. Musa Anter]"
1,"Doctor David Coleman, as the PC, I like that d...","[Doctor David Coleman, PA Kazick Downski]"
2,Kazim Kurdi as her PCP is very famous.,[]
3,PA Adam Smith drives a lot of different cars w...,[PA Adam Smith]
4,PA Ebbie Dark always gets up early. She loves ...,"[PA Ebbie Dark, PA Foresnjs Adam]"
5,"Primary care provider, Sara-Lopez Mary-Colly, ...","[provider, Sara-Lopez Mary, Dr. Alex Ferguson,..."
6,"Orthopedic surgeon, Lopez C. Tunner, MD, (PID:...","[surgeon, Lopez C. , Doctor Koray Peksever]"
7,"Endocrinologist, Gerry Samantha, NP, (PID: 243...","[ist, Gerry Samantha, Dr. Sarah Goray, PA Fore..."
8,"Podiatrist, Roy Rolls (NPI::23426234398) lets ...","[ist, Roy Rolls]"
9,"In the morning MD, Shiv Reed, PA-C, (PID:24243...","[MD, Shiv Reed]"


In [121]:
df[df.index == 10].Providers3.unique


<bound method Series.unique of 10    [Dr. Scott Hahaha , Trump Boyler, NP]
Name: Providers3, dtype: object>

In [122]:
df.to_excel("test1.xlsx")